In [4]:
!pip install tensorflow opencv-python mediapipe sklearn matplotlib

Defaulting to user installation because normal site-packages is not writeable


In [5]:
!pip install sklearn

Defaulting to user installation because normal site-packages is not writeable


In [6]:
!pip install tensorflow-gpu

Defaulting to user installation because normal site-packages is not writeable
  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  ERROR: Error [WinError 225] Operation did not complete successfully because the file contains a virus or potentially unwanted software while executing command python setup.py egg_info
ERROR: Could not install packages due to an OSError: [WinError 225] Operation did not complete successfully because the file contains a virus or potentially unwanted software



In [7]:
!pip install mediapipe

Defaulting to user installation because normal site-packages is not writeable


In [8]:
!pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable


In [9]:
!pip install sklearn matplotlib

Defaulting to user installation because normal site-packages is not writeable


In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [2]:
mp_holistic = mp.solutions.holistic    # Holistic Model
mp_drawing = mp.solutions.drawing_utils    # Drawing Utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [5]:
def draw_styled_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                             mp_drawing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),
                              mp_drawing.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1)
                             )
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80, 22, 10), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2)
                             )
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(121, 44, 250), thickness=2, circle_radius=2)
                             )
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245, 137, 66), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(245, 66, 250), thickness=2, circle_radius=2)
                             )

In [15]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()

        image, results = mediapipe_detection(frame, holistic)
        
        draw_styled_landmarks(image, results)

        cv2.imshow('OpenCV Feed', image)
        if cv2.waitKey(10) & 0xFF== ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [6]:
len(results.left_hand_landmarks.landmark)

NameError: name 'results' is not defined

In [21]:
draw_landmarks(frame, results)

In [ ]:
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

In [7]:
results

NameError: name 'results' is not defined

In [8]:
all_landmarks = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    all_landmarks.append(test)

NameError: name 'results' is not defined

In [6]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    
    return np.concatenate([pose, face, lh, rh])

In [10]:
extract_keypoints(results).shape

NameError: name 'results' is not defined

In [7]:
DATA_PATH = os.path.join("MP_Data")
actions = np.array(['hello', 'thanks', 'iloveyou', 'what', 'bad'])
no_sequences = 30
sequence_length = 30

In [8]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [54]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for action in actions:
        for sequence in range(no_sequences):
            for frame_num in range(sequence_length):
                ret, frame = cap.read()

                image, results = mediapipe_detection(frame, holistic)

                draw_styled_landmarks(image, results)
                
                if frame_num == 0:
                    cv2.putText(image, 'Starting Collection', (120, 200),
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting Frames for {} Video Number {}'.format(action, sequence), (15, 12),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.waitKey(2000)
                else:
                    cv2.putText(image, 'Collecting frames for {} Video NUmber {}'.format(action, sequence), (15, 12),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)

                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)
                
                cv2.imshow('OpenCV Feed', image)
                
                if cv2.waitKey(10) & 0xFF== ord('q'):
                    break
cap.release()
cv2.destroyAllWindows()

In [ ]:
cap.release()
cv2.destroyAllWindows()

In [13]:
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable


In [9]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [10]:
label_map = {label:num for num, label in enumerate(actions)}

In [11]:
label_map

{'hello': 0, 'thanks': 1, 'iloveyou': 2, 'what': 3, 'bad': 4}

In [12]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [13]:
X = np.array(sequences)

In [14]:
y = to_categorical(labels).astype(int)

In [15]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [16]:
x_train

array([[[ 6.26353860e-01,  4.82843518e-01, -6.85427845e-01, ...,
          4.23383743e-01,  7.45212376e-01, -6.99213222e-02],
        [ 6.26374125e-01,  4.81935591e-01, -6.86214089e-01, ...,
          4.38114405e-01,  7.66784489e-01, -7.34383315e-02],
        [ 6.26429737e-01,  4.81764108e-01, -6.60569251e-01, ...,
          4.39368278e-01,  7.61855721e-01, -6.87645599e-02],
        ...,
        [ 6.28538549e-01,  4.85320181e-01, -6.55752301e-01, ...,
          4.43711460e-01,  7.53784955e-01, -6.64444193e-02],
        [ 6.28433824e-01,  4.85318124e-01, -6.91034436e-01, ...,
          4.45230752e-01,  7.50695288e-01, -6.58001751e-02],
        [ 6.27745986e-01,  4.85329002e-01, -6.47605836e-01, ...,
          4.41361457e-01,  7.56746173e-01, -6.53130487e-02]],

       [[ 5.43494344e-01,  4.94294375e-01, -5.64222038e-01, ...,
          5.84061623e-01,  5.79558611e-01, -2.80067734e-02],
        [ 5.44563830e-01,  4.94180411e-01, -5.48909605e-01, ...,
          5.88437378e-01,  5.81551671e

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [18]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [19]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30, 1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [20]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [21]:
model.fit(x_train, y_train, epochs=100, callbacks=[tb_callback])

Epoch 1/100
5/5 [==============================] - 7s 60ms/step - loss: 3.1375 - categorical_accuracy: 0.2042
Epoch 2/100
5/5 [==============================] - 0s 58ms/step - loss: 2.9208 - categorical_accuracy: 0.2183
Epoch 3/100
5/5 [==============================] - 0s 60ms/step - loss: 5.6231 - categorical_accuracy: 0.1761
Epoch 4/100
5/5 [==============================] - 0s 61ms/step - loss: 11.5883 - categorical_accuracy: 0.1972
Epoch 5/100
5/5 [==============================] - 0s 60ms/step - loss: 9.9355 - categorical_accuracy: 0.1690
Epoch 6/100
5/5 [==============================] - 0s 64ms/step - loss: 16.4013 - categorical_accuracy: 0.1901
Epoch 7/100
5/5 [==============================] - 0s 61ms/step - loss: 28.9818 - categorical_accuracy: 0.1761
Epoch 8/100
5/5 [==============================] - 0s 60ms/step - loss: 12.4744 - categorical_accuracy: 0.1831
Epoch 9/100
5/5 [==============================] - 0s 61ms/step - loss: 15.9063 - categorical_accuracy: 0.1901
Epoch

5/5 [==============================] - 0s 64ms/step - loss: 6.3363 - categorical_accuracy: 0.1831
Epoch 75/100
5/5 [==============================] - 0s 63ms/step - loss: 7.9934 - categorical_accuracy: 0.1690
Epoch 76/100
5/5 [==============================] - 0s 64ms/step - loss: 6.6036 - categorical_accuracy: 0.1831
Epoch 77/100
5/5 [==============================] - 0s 62ms/step - loss: 5.7434 - categorical_accuracy: 0.1690
Epoch 78/100
5/5 [==============================] - 0s 65ms/step - loss: 5.4540 - categorical_accuracy: 0.2113
Epoch 79/100
5/5 [==============================] - 0s 62ms/step - loss: 7.9749 - categorical_accuracy: 0.1408
Epoch 80/100
5/5 [==============================] - 0s 63ms/step - loss: 5.5438 - categorical_accuracy: 0.2394
Epoch 81/100
5/5 [==============================] - 0s 63ms/step - loss: 6.2587 - categorical_accuracy: 0.1690
Epoch 82/100
5/5 [==============================] - 0s 62ms/step - loss: 5.8940 - categorical_accuracy: 0.2042
Epoch 83/100
5

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 5)                 165       
                                                                 
Total params: 596741 (2.28 MB)
Trainable params: 596741 

In [23]:
res = model.predict(x_test)

1/1 [==============================] - 1s 519ms/step


In [24]:
actions[np.argmax(res[6])]

'what'

In [25]:
model.save('action.keras')

In [26]:
model.load_weights('action.keras')

In [27]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [28]:
yhat = model.predict(x_train)

5/5 [==============================] - 0s 20ms/step


In [29]:
ytrue = np.argmax(y_train, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [30]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[114,   0],
        [ 28,   0]],

       [[114,   0],
        [ 28,   0]],

       [[112,   0],
        [ 30,   0]],

       [[  0, 115],
        [  0,  27]],

       [[113,   0],
        [ 29,   0]]], dtype=int64)

In [31]:
accuracy_score(ytrue, yhat)

0.19014084507042253

In [32]:
accuracy_score(ytrue, yhat)

0.19014084507042253

In [33]:
sequence = []
sentence = []
threshold = 0.8

# Load your trained model here
# model = load_your_model_here()

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()

        image, results = mediapipe_detection(frame, holistic)
        draw_styled_landmarks(image, results)

        keypoints = extract_keypoints(results)
        sequence.insert(0, keypoints)
        sequence = sequence[:30]

        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])

            if np.any(res > threshold):
                if len(sentence) > 0:
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

        if len(sentence) > 5:
            sentence = sentence[-5:]

        cv2.rectangle(image, (0, 0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        cv2.imshow('OpenCV Feed', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 30ms/step
what
1/1 [==============================] - 0s 20ms/step
what
1/1 [==============================] - 0s 24ms/step
what
1/1 [==============================] - 0s 19ms/step
what
1/1 [==============================] - 0s 24ms/step
what
1/1 [==============================] - 0s 27ms/step
what
1/1 [==============================] - 0s 22ms/step
what
1/1 [==============================] - 0s 19ms/step
what
1/1 [==============================] - 0s 28ms/step
what
1/1 [==============================] - 0s 21ms/step
what
1/1 [==============================] - 0s 28ms/step
what
1/1 [==============================] - 0s 30ms/step
what
1/1 [==============================] - 0s 25ms/step
what
1/1 [==============================] - 0s 22ms/step
what
1/1 [==============================] - 0s 31ms/step
what
1/1 [==============================] - 0s 26ms/step
what
1/1 [==============================] - 0s 27ms/step
what
1/1 [==========================

1/1 [==============================] - 0s 19ms/step
what
1/1 [==============================] - 0s 18ms/step
what
1/1 [==============================] - 0s 24ms/step
what
1/1 [==============================] - 0s 24ms/step
what
1/1 [==============================] - 0s 27ms/step
what
1/1 [==============================] - 0s 19ms/step
what
1/1 [==============================] - 0s 19ms/step
what
1/1 [==============================] - 0s 23ms/step
what
1/1 [==============================] - 0s 21ms/step
what
1/1 [==============================] - 0s 24ms/step
what
1/1 [==============================] - 0s 23ms/step
what
1/1 [==============================] - 0s 24ms/step
what
1/1 [==============================] - 0s 22ms/step
what
1/1 [==============================] - 0s 30ms/step
what
1/1 [==============================] - 0s 27ms/step
what
1/1 [==============================] - 0s 23ms/step
what
1/1 [==============================] - 0s 22ms/step
what
1/1 [==========================

what
1/1 [==============================] - 0s 19ms/step
what
1/1 [==============================] - 0s 24ms/step
what
1/1 [==============================] - 0s 26ms/step
what
1/1 [==============================] - 0s 26ms/step
what
1/1 [==============================] - 0s 18ms/step
what
1/1 [==============================] - 0s 18ms/step
what
1/1 [==============================] - 0s 32ms/step
what
1/1 [==============================] - 0s 27ms/step
what
1/1 [==============================] - 0s 20ms/step
what
1/1 [==============================] - 0s 21ms/step
what
1/1 [==============================] - 0s 20ms/step
what
1/1 [==============================] - 0s 17ms/step
what
1/1 [==============================] - 0s 22ms/step
what
1/1 [==============================] - 0s 27ms/step
what
1/1 [==============================] - 0s 37ms/step
what
1/1 [==============================] - 0s 20ms/step
what
1/1 [==============================] - 0s 16ms/step
what
1/1 [=====================

1/1 [==============================] - 0s 19ms/step
what
1/1 [==============================] - 0s 28ms/step
what
1/1 [==============================] - 0s 27ms/step
what
1/1 [==============================] - 0s 29ms/step
what
1/1 [==============================] - 0s 30ms/step
what
1/1 [==============================] - 0s 19ms/step
what
1/1 [==============================] - 0s 19ms/step
what
1/1 [==============================] - 0s 27ms/step
what
1/1 [==============================] - 0s 31ms/step
what
1/1 [==============================] - 0s 18ms/step
what
1/1 [==============================] - 0s 35ms/step
what
1/1 [==============================] - 0s 19ms/step
what
1/1 [==============================] - 0s 16ms/step
what
1/1 [==============================] - 0s 28ms/step
what
1/1 [==============================] - 0s 17ms/step
what
1/1 [==============================] - 0s 20ms/step
what
1/1 [==============================] - 0s 16ms/step
what
1/1 [==========================

1/1 [==============================] - 0s 21ms/step
what
1/1 [==============================] - 0s 14ms/step
what
1/1 [==============================] - 0s 26ms/step
what
1/1 [==============================] - 0s 20ms/step
what
1/1 [==============================] - 0s 17ms/step
what
1/1 [==============================] - 0s 24ms/step
what
1/1 [==============================] - 0s 26ms/step
what
1/1 [==============================] - 0s 33ms/step
what
1/1 [==============================] - 0s 29ms/step
what
1/1 [==============================] - 0s 22ms/step
what
1/1 [==============================] - 0s 17ms/step
what
1/1 [==============================] - 0s 29ms/step
what
1/1 [==============================] - 0s 19ms/step
what
1/1 [==============================] - 0s 21ms/step
what
1/1 [==============================] - 0s 33ms/step
what
1/1 [==============================] - 0s 19ms/step
what
1/1 [==============================] - 0s 30ms/step
what
1/1 [==========================

KeyboardInterrupt: 

In [ ]:
cap.release()
cv2.destroyAllWindows()

In [ ]:
res[np.argmax(res)] > threshold

In [ ]:
(num_sequences,30,1662)

In [ ]:
model.predict(np.expand_dims(X_test[0], axis=0))